In [1]:
import tensorflow as tf
from keras import layers

MODEL_PATH = "C:/code/potato_disease/Models/potato_disease_model/3"

# ✅ Use TFSMLayer to load the model
model = layers.TFSMLayer(MODEL_PATH, call_endpoint="serving_default")

print("Model loaded successfully using TFSMLayer!")


Model loaded successfully using TFSMLayer!


In [ ]:
import nest_asyncio
nest_asyncio.apply()
import asyncio
import uvicorn
from fastapi import FastAPI, File, UploadFile
from fastapi.middleware.cors import CORSMiddleware
import numpy as np
from io import BytesIO
from PIL import Image
import tensorflow as tf
from keras import layers

# ✅ Patch event loop to fix async errors in Jupyter Notebook
nest_asyncio.apply()

app = FastAPI()

# Enable CORS for frontend requests
origins = ["http://localhost", "http://localhost:3000"]
app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# ✅ Load the model using TFSMLayer
MODEL_PATH = "C:/code/potato_disease/Models/potato_disease_model/3"
model = layers.TFSMLayer(MODEL_PATH, call_endpoint="serving_default")

CLASS_NAMES = ["Potato___Early_blight", "Potato___Late_blight", "Potato___healthy"]

@app.get("/ping")
async def ping():
    return "Hello, I am alive"

def read_file_as_image(data) -> np.ndarray:
    image = np.array(Image.open(BytesIO(data)).resize((256, 256)))  # Resize to match model input
    image = image / 255.0  # Normalize
    return image

@app.post("/predict")
async def predict(file: UploadFile = File(...)):
    image = read_file_as_image(await file.read())
    img_batch = np.expand_dims(image, 0)  # Add batch dimension

    # ✅ Use TFSMLayer for inference
    prediction = model(img_batch)

    # ✅ Extract numerical values from dictionary output
    prediction = prediction["output_0"].numpy()  # Extract output from dict
    
    predicted_class = CLASS_NAMES[np.argmax(prediction)]
    confidence = float(np.max(prediction))

    return {
        "class": predicted_class,
        "confidence": confidence
    }

# ✅ Run FastAPI app inside Jupyter Notebook
uvicorn.run(app, host="localhost", port=8014)


INFO:     Started server process [25576]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://localhost:8014 (Press CTRL+C to quit)


INFO:     ::1:52653 - "POST /predict HTTP/1.1" 200 OK
INFO:     ::1:52753 - "POST /predict HTTP/1.1" 200 OK
INFO:     ::1:52761 - "POST /predict HTTP/1.1" 200 OK
INFO:     ::1:52822 - "POST /predict HTTP/1.1" 200 OK
INFO:     ::1:52826 - "POST /predict HTTP/1.1" 500 Internal Server Error


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "C:\Users\raman\anaconda3\Lib\site-packages\uvicorn\protocols\http\h11_impl.py", line 403, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\raman\anaconda3\Lib\site-packages\uvicorn\middleware\proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\raman\anaconda3\Lib\site-packages\fastapi\applications.py", line 1054, in __call__
    await super().__call__(scope, receive, send)
  File "C:\Users\raman\anaconda3\Lib\site-packages\starlette\applications.py", line 112, in __call__
    await self.middleware_stack(scope, receive, send)
  File "C:\Users\raman\anaconda3\Lib\site-packages\starlette\middleware\errors.py", line 187, in __call__
    raise exc
  File "C:\Users\raman\anaconda3\Lib\site-packages\starlette\midd

INFO:     ::1:53681 - "POST /predict HTTP/1.1" 200 OK
INFO:     ::1:53757 - "POST /predict HTTP/1.1" 200 OK
